In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
from builtins import range

import matplotlib as mpl
import numpy as np

from blond.beam.beam import Beam, Proton
from blond.beam.distributions import bigaussian, parabolic
from blond.beam.profile import CutOptions, FitOptions, Profile
from blond.input_parameters.rf_parameters import RFStation
#  BLonD Imports
from blond.input_parameters.ring import Ring
from blond.monitors.monitors import BunchMonitor
from blond.plots.plot import Plot
from blond.trackers.tracker import RingAndRFTracker

from blond.utils.assembler import Assembler, Trackable

import logging
logging.basicConfig(level=logging.DEBUG)

In [12]:

# Simulation parameters -------------------------------------------------------
# Bunch parameters
N_b = 1e9           # Intensity
N_p = 50000         # Macro-particles
tau_0 = 0.4e-9          # Initial bunch length, 4 sigma [s]

# Machine and RF parameters
C = 26658.883        # Machine circumference [m]
p_i = 450e9         # Synchronous momentum [eV/c]
p_f = 460.005e9      # Synchronous momentum, final
h = 35640            # Harmonic number
V = 6e6                # RF voltage [V]
dphi = 0             # Phase modulation/offset
gamma_t = 55.759505  # Transition gamma
alpha = 1. / gamma_t / gamma_t        # First order mom. comp. factor

# Tracking details
N_t = 2000           # Number of turns to track
dt_plt = 200         # Time steps between plots


# Simulation setup ------------------------------------------------------------
print("Setting up the simulation...")
print("")


# Define general parameters
ring = Ring(C, alpha, np.linspace(p_i, p_f, 2001), Proton(), N_t)

# Define beam and distribution
beam = Beam(ring, N_p, N_b)


# Define RF station parameters and corresponding tracker
rf = RFStation(ring, [h], [V], [dphi])
long_tracker = RingAndRFTracker(rf, beam)


bigaussian(ring, rf, beam, tau_0 / 4, reinsertion=True, seed=1)
# parabolic(ring, rf, beam, tau_0, seed=1)


# Need slices for the Gaussian fit
profile = Profile(beam, CutOptions(n_slices=100),
                  FitOptions(fit_option='gaussian'))


Setting up the simulation...



In [19]:
# Use case 1: Initialize from object list
assembler = Assembler(element_list=[ring, beam, rf, long_tracker, profile])
assembler.build_pipeline()
print(assembler)

DEBUG:blond.utils.assembler:Error importing module blond.gpu.butils_wrap_cupy: No module named 'cupy'
DEBUG:blond.utils.assembler:Error importing module blond.toolbox.parameter_scaling: /home/kiliakis/git/BLonD-1/venv/lib/python3.9/site-packages/PyQt5/QtCore.abi3.so: undefined symbol: _ZdaPvm, version Qt_5
DEBUG:blond.utils.assembler:Error importing module blond.utils.mpi_config: No module named 'mpi4py'
DEBUG:blond.utils.assembler:Found element of type: Ring
DEBUG:blond.utils.assembler:Found element of type: Beam
DEBUG:blond.utils.assembler:Found element of type: RFStation
DEBUG:blond.utils.assembler:Found element of type: RingAndRFTracker
DEBUG:blond.utils.assembler:Found element of type: Profile


Assembler with 2 tracking elements and 5 objects.
Pipeline elements:
	0. blond.beam.profile
	1. blond.trackers.tracker
Objects:
	Ring
	Beam
	RFStation
	RingAndRFTracker
	Profile


In [14]:
# timing.reset()
assembler.track(100, with_timing=True)
assembler.report_timing()

+------------------------+------------------+--------------------+---------+-------+------------+
| function               | total time (sec) | time per call (ms) | std (%) | calls | global (%) |
+------------------------+------------------+--------------------+---------+-------+------------+
| blond.beam.profile     | 0.043            | 0.428              | 32.392  | 100   | 38.379     |
| blond.trackers.tracker | 0.069            | 0.687              | 14.194  | 100   | 61.621     |
| other                  | 0.000            | 0.000              | 0.000   | 1     | 0.000      |
| total_time             | 0.111            | 111.432            | 0.000   | 1     | 100        |
+------------------------+------------------+--------------------+---------+-------+------------+


In [91]:
# Use case 2: Initialize from dictionary
assembler2 = Assembler(element_list=[
    {'Ring': ((C, alpha), {'synchronous_data': np.linspace(p_i, p_f, 2001), 'Particle': Proton(), 'n_turns': N_t})},
    {'Beam': ('Ring', N_p, N_b)},
    {'RFStation': {'Ring': 'Ring', 'harmonic': [h], 'voltage': [V], 'phi_rf_d': [dphi]}},
    {'RingAndRFTracker': {'RFStation': 'RFStation', 'Beam': 'Beam'}},
    {'Profile': {'Beam': 'Beam', 'CutOptions': CutOptions(n_slices=100), 'FitOptions': FitOptions(fit_option='gaussian')}}
])

assembler2.build_pipeline()
assembler2.build_distribution(distribution={
    'bigaussian': {'Ring': 'Ring', 'RFStation': 'RFStation', 'Beam': 'Beam', 'sigma_dt': tau_0 / 4,
                   'reinsertion': True, 'seed': 1}
})

print(assembler2.pipeline)
print(assembler2.objects)

Error importing module blond.gpu.butils_wrap_cupy: No module named 'cupy'
Error importing module blond.toolbox.parameter_scaling: /home/kiliakis/git/BLonD-1/venv/lib/python3.9/site-packages/PyQt5/QtCore.abi3.so: undefined symbol: _ZdaPvm, version Qt_5
Error importing module blond.utils.mpi_config: No module named 'mpi4py'
Found dictionary element with 2 positional arguments and 3 kwargs
Found element of type: Ring
Found dictionary element with 3 positional arguments and 0 kwargs
Found element of type: Beam
Found dictionary element with 0 positional arguments and 4 kwargs
Found element of type: RFStation
Found dictionary element with 0 positional arguments and 2 kwargs
Found element of type: RingAndRFTracker
Found dictionary element with 0 positional arguments and 3 kwargs
Found element of type: Profile
Found dictionary element with 0 positional arguments and 6 kwargs
[<bound method RingAndRFTracker.track of <blond.trackers.tracker.RingAndRFTracker object at 0x7fbd9d38bfa0>>, <bound met

/home/kiliakis/git/BLonD-1/blond/toolbox/filters_and_fitting.py:124: RuntimeWarning: invalid value encountered in divide
  return A * np.exp(-(x - x0)**2 / 2. / sx**2)
/home/kiliakis/git/BLonD-1/blond/toolbox/filters_and_fitting.py:124: RuntimeWarning: divide by zero encountered in divide
  return A * np.exp(-(x - x0)**2 / 2. / sx**2)


In [92]:
# print(assembler2.pipeline[0])
# type(assembler2.pipeline[0].__func__.__str__())
# print(assembler2.pipeline[0].__func__.__globals__['__name__'])
# inspect.getmembers(assembler2.pipeline[0].__func__)

In [94]:
print(assembler2.Beam)

In [93]:
custom_order = ['Ring', 'Beam', 'RFStation', 'Profile', 'SynchrotronRadiation', 'Feedbacks', 'RingAndRFTracker']

In [ ]:
print(timing.report())

In [ ]:
timing.times

In [6]:
import numpy as np
import inspect

inspect.getmembers(np, inspect.isfunction)


[('__dir__', <function numpy.__dir__()>),
 ('__getattr__', <function numpy.__getattr__(attr)>),
 ('_no_nep50_warning', <function numpy._no_nep50_warning()>),
 ('_pyinstaller_hooks_dir', <function numpy._pyinstaller_hooks_dir()>),
 ('add_newdoc',
  <function numpy.core.function_base.add_newdoc(place, obj, doc, warn_on_python=True)>),
 ('all',
  <function numpy.all(a, axis=None, out=None, keepdims=<no value>, *, where=<no value>)>),
 ('allclose',
  <function numpy.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)>),
 ('alltrue', <function numpy.alltrue(*args, **kwargs)>),
 ('amax',
  <function numpy.amax(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>),
 ('amin',
  <function numpy.amin(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>),
 ('angle', <function numpy.angle(z, deg=False)>),
 ('any',
  <function numpy.any(a, axis=None, out=None, keepdims=<no value>, *, where=<no value>)>),
 ('append', <function numpy.a